In [1]:
import polars as pl
import pysentiment2 as ps
import math
import os
import re
from matplotlib_venn import venn2, venn3
import urllib.parse

In [2]:
lm_dict = pl.read_csv(r'C:\Users\310\Desktop\Progects_Py\Parsim-sec\src\Analysis\Loughran-McDonald_MasterDictionary_1993-2021.csv')
hv_dict = pl.read_excel(r'C:\Users\310\Desktop\Progects_Py\Parsim-sec\src\converter_api\Harvard_inquirerbasic.xls')
hv_dict.sample(10)

Entry,Source,Positiv,Negativ,Pstv,Affil,Ngtv,Hostile,Strong,Power,Weak,Submit,Active,Passive,Pleasur,Pain,Feel,Arousal,EMOT,Virtue,Vice,Ovrst,Undrst,Academ,Doctrin,Econ@,Exch,ECON,Exprsv,Legal,Milit,Polit@,POLIT,Relig,Role,COLL,Work,…,WltTran,WltOth,WltTot,WlbGain,WlbLoss,WlbPhys,WlbPsyc,WlbPt,WlbTot,EnlGain,EnlLoss,EnlEnds,EnlPt,EnlOth,EnlTot,SklAsth,SklPt,SklOth,SklTot,TrnGain,TrnLoss,TranLw,MeansLw,EndsLw,ArenaLw,PtLw,Nation,Anomie,NegAff,PosAff,SureLw,If,NotLw,TimeSpc,FormLw,Othtags,Defined
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,null,str,str,str,str,str,str,str,str,str,str,…,str,str,str,str,str,str,str,str,str,str,str,null,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""SHELTER#1""","""H4Lvd""","""Positiv""",null,"""Pstv""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,"""WlbPhys""",null,null,"""WlbTot""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""Noun""","""|"""
"""EARN#1""","""H4Lvd""",null,null,null,null,null,null,"""Strong""",null,null,null,"""Active""",null,null,null,null,null,null,null,null,null,null,null,null,"""Econ@""",null,null,null,null,null,null,null,null,null,null,"""Work""",…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""SUPV""","""| 72% verb: To…"
"""STAGE#1""","""H4Lvd""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""Noun""","""| 68% noun: A …"
"""SIXTH#1""","""H4""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""|"""
"""DIFFERENT#2""","""H4Lvd""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""Modif""","""| 3% adverb: ""…"
"""INCH""","""H4Lvd""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""TimeSpc""",null,"""Noun""","""|"""
"""TRANSMIT""","""Lvd""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""TranLw""",null,null,null,null,null,null,null,null,null,null,null,null,null,"""SUPV""",null
"""HURRY#2""","""H4Lvd""",null,null,null,null,null,null,null,null,null,null,"""Active""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,

In [3]:
#ensure that harvard dictionary entries are compatable with input strings
words = hv_dict.select(pl.col('Entry'))

hv_words = words.to_series().str.to_lowercase()

hv_words = hv_words.map_elements(lambda word: re.sub(r'#\d+', '', word))

hv_dict = hv_dict.replace_column(0, hv_words)

hv_dict = hv_dict.unique(subset="Entry", keep="last")


In [5]:
df_dict = {}

path = r'C:\Users\310\Desktop\Progects_Py\data\Parsim_sec_data\prepared_data\2013_reports'

for company_name in os.listdir(path):
    
    file_path = os.path.join(path, company_name)

    match = re.search(r"\\([A-Z]+(?:\.[A-Z])?)_reports\.parquet", file_path)

    if match:
        ticker_symbol = match.group(1)
        print("Ticker Symbol:", ticker_symbol, '\n', file_path)
    else:
        print(f"Ticker symbol for {company_name} is not found")

    df_name = f"{ticker_symbol}"
    df = pl.read_parquet(file_path)

    df_dict[df_name] = df

Ticker Symbol: AAPL 
 C:\Users\310\Desktop\Progects_Py\data\Parsim_sec_data\prepared_data\2013_reports\AAPL_reports.parquet
Ticker Symbol: ABBV 
 C:\Users\310\Desktop\Progects_Py\data\Parsim_sec_data\prepared_data\2013_reports\ABBV_reports.parquet
Ticker Symbol: ABT 
 C:\Users\310\Desktop\Progects_Py\data\Parsim_sec_data\prepared_data\2013_reports\ABT_reports.parquet
Ticker Symbol: ACGL 
 C:\Users\310\Desktop\Progects_Py\data\Parsim_sec_data\prepared_data\2013_reports\ACGL_reports.parquet
Ticker Symbol: ACN 
 C:\Users\310\Desktop\Progects_Py\data\Parsim_sec_data\prepared_data\2013_reports\ACN_reports.parquet
Ticker Symbol: ADBE 
 C:\Users\310\Desktop\Progects_Py\data\Parsim_sec_data\prepared_data\2013_reports\ADBE_reports.parquet
Ticker Symbol: ADI 
 C:\Users\310\Desktop\Progects_Py\data\Parsim_sec_data\prepared_data\2013_reports\ADI_reports.parquet
Ticker Symbol: ADM 
 C:\Users\310\Desktop\Progects_Py\data\Parsim_sec_data\prepared_data\2013_reports\ADM_reports.parquet
Ticker Symbol: A

In [7]:
df_dict['AAPL']

2009-01-23_26418,2009-04-23_27638,2009-07-22_27934,2009-10-27_43245,2010-01-25_26407,2010-04-21_26069,2010-07-21_26743,2010-10-27_37434,2011-01-19_20797,2011-04-21_21532,2011-07-20_23896,2011-10-26_33819,2012-01-25_19880,2012-04-25_20210,2012-07-25_20442,2012-10-31_33824,2013-01-24_20278,2013-04-24_21693,2013-07-24_22519,2013-10-30_35647
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""company""","""company""","""company""","""delinquent""","""company""","""company""","""company""","""delinquent""","""company""","""company""","""company""","""delinquent""","""company""","""company""","""company""","""delinquent""","""company""","""company""","""company""","""delinquent"""
"""company""","""company""","""company""","""best""","""company""","""company""","""company""","""best""","""company""","""company""","""company""","""best""","""company""","""company""","""company""","""best""","""company""","""company""","""company""","""best"""
"""shell""","""shell""","""shell""","""knowledge""","""shell""","""shell""","""shell""","""knowledge""","""shell""","""shell""","""shell""","""knowledge""","""shell""","""shell""","""shell""","""knowledge""","""shell""","""shell""","""shell""","""knowledge"""
"""company""","""company""","""company""","""definitive""","""company""","""company""","""company""","""definitive""","""company""","""company""","""company""","""definitive""","""company""","""company""","""company""","""definitive""","""company""","""company""","""company""","""definitive"""
"""common""","""common""","""common""","""company""","""common""","""common""","""common""","""company""","""common""","""common""","""common""","""company""","""common""","""common""","""common""","""company""","""common""","""common""","""common""","""company"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""N/A""","""N/A""","""N/A""","""behalf""","""N/A""","""N/A""","""N/A""","""N/A""","""N/A""","""N/A""","""N/A""","""N/A""","""N/A""","""N/A""","""N/A""","""N/A""","""N/A""","""N/A""","""N/A""","""N/A"""
"""N/A""","""N/A""","""N/A""","""jointly""","""N/A""","""N/A""","""N/A""","""N/A""","""N/A""","""N/A""","""N/A""","""N/A""","""N/A""","""N/A""","""N/A""","""N/A""","""N/A""","""N/A""","""N/A""","""N/A"""
"""N/A""","""N/A""","""N/A""","""substitution""","""N/A""","""N/A""","""N/A""","""N/A""","""N/A""","""N/A""","""N/A""","""N/A""","""N/A""","""N/A""","""N/A""","""N/A""","""N/A""","""N/A""","""N/A""","""N/A"""


In [8]:
total_num_of_rep = 0

for value in df_dict.values():
    total_num_of_rep += value.width
print(total_num_of_rep)

8512


In [9]:
total_num_of_words = 0

for value in df_dict.values():
    for col in value:
        total_num_of_words += col.count()

print(total_num_of_words)

35222298


In [10]:
#average doc length
average_word_count = total_num_of_words/total_num_of_rep
print(f'Average doc length in words: {round(average_word_count)}')
ln_average_word_count = math.log(average_word_count)

Average doc length in words: 4138


In [11]:
#ENSURE CASE CoMpAtAbIlItY
positive_words_lm = lm_dict.filter(lm_dict["Positive"] > 0).to_series().str.to_lowercase()
negative_words_lm = lm_dict.filter(lm_dict["Negative"] > 0).to_series().str.to_lowercase()

print(len(positive_words_lm) + len(negative_words_lm))

positive_words_hv = hv_dict.filter(hv_dict["Positiv"] == 'Positiv').to_series().str.to_lowercase()
negative_words_hv = hv_dict.filter(hv_dict["Negativ"] == 'Negativ').to_series().str.to_lowercase()

print(len(positive_words_hv) + len(negative_words_hv))


2692
3475


In [12]:
#example series 
example_series = df_dict['ALGN'][df_dict['ALGN'].columns[1]]
example_df = df_dict['ALGN']
example_df


2009-02-27_38762,2009-05-07_18652,2009-08-05_19064,2009-11-05_21641,2010-02-26_35302,2010-05-06_14646,2010-08-05_15380,2010-11-04_15834,2011-02-24_33539,2011-05-05_15378,2011-08-08_19067,2011-11-04_18847,2012-02-29_36544,2012-05-08_15925,2012-08-02_16086,2012-11-09_18553,2013-03-01_40579,2013-05-03_18085,2013-08-02_19336,2013-11-01_19799
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""delinquent""","""principal""","""principal""","""principal""","""principal""","""principal""","""principal""","""principal""","""principal""","""principal""","""principal""","""principal""","""principal""","""principal""","""principal""","""principal""","""principal""","""principal""","""principal""","""principal"""
"""best""","""company""","""company""","""company""","""delinquent""","""company""","""company""","""company""","""delinquent""","""company""","""company""","""company""","""delinquent""","""company""","""company""","""company""","""delinquent""","""company""","""company""","""company"""
"""knowledge""","""shell""","""company""","""company""","""best""","""company""","""company""","""company""","""best""","""company""","""company""","""company""","""best""","""company""","""company""","""company""","""best""","""company""","""company""","""company"""
"""definitive""","""company""","""shell""","""shell""","""knowledge""","""shell""","""shell""","""shell""","""knowledge""","""shell""","""shell""","""shell""","""knowledge""","""shell""","""shell""","""shell""","""knowledge""","""shell""","""shell""","""shell"""
"""company""","""outstanding""","""company""","""company""","""definitive""","""company""","""company""","""company""","""definitive""","""company""","""company""","""company""","""definitive""","""company""","""company""","""company""","""definitive""","""company""","""company""","""company"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""N/A""","""N/A""","""N/A""","""N/A""","""N/A""","""N/A""","""N/A""","""N/A""","""N/A""","""N/A""","""N/A""","""N/A""","""N/A""","""N/A""","""N/A""","""N/A""","""omitted""","""N/A""","""N/A""","""N/A"""
"""N/A""","""N/A""","""N/A""","""N/A""","""N/A""","""N/A""","""N/A""","""N/A""","""N/A""","""N/A""","""N/A""","""N/A""","""N/A""","""N/A""","""N/A""","""N/A""","""behalf""","""N/A""","""N/A""","""N/A"""
"""N/A""","""N/A""","""N/A""","""N/A""","""N/A""","""N/A""","""N/A""","""N/A""","""N/A""","""N/A""","""N/A""","""N/A""","""N/A""","""N/A""","""N/A""","""N/A""","""substitution""","""N/A""","""N/A""","""N/A"""


This is the func, that appled to a series and get sentiment words from it. As an output we get a df with first col - grouped words that appeaps in LOUGHRAN dictionary and has >0 value in "positive" or "negative" cols. Secon col contain count of corresponding word in this document. Third col contain boolean, indicatin if this word has positive sentiment (True) or negative (False)

In [13]:
def count_sentiment_w_in_doc(series):
        
        #grouped series that became a df, where each row is unique word and its count in document 
        num_of_w = series.value_counts().rename({"count": "count in doc"})

        #print(num_of_w.shape)
        
        #first col of this df (only words by itself)
        only_w = num_of_w[num_of_w.columns[0]]
        

        #Filterd df, there is only words that have positive or negative sentiment score in this document 
        pos_w_lm = num_of_w.filter(only_w.is_in(positive_words_lm)).with_columns(pl.lit(True).alias("positive_lm"))
        neg_w_lm = num_of_w.filter(only_w.is_in(negative_words_lm)).with_columns(pl.lit(False).alias("positive_lm"))
        #print(pos_w_lm.shape[0], neg_w_lm.shape[0])

        pos_w_hv = num_of_w.filter(only_w.is_in(positive_words_hv)).with_columns(pl.lit(True).alias("positive_hv"))
        neg_w_hv = num_of_w.filter(only_w.is_in(negative_words_hv)).with_columns(pl.lit(False).alias("positive_hv"))
        #print(pos_w_hv.shape[0], neg_w_hv.shape[0])

        #Staked for one df for further operations 
        df_stacked_lm = neg_w_lm.vstack(pos_w_lm)
        df_stacked_hv = neg_w_hv.vstack(pos_w_hv)
        #print(f'df_stacked_lm: {df_stacked_lm.shape}; df_stacked_hv: {df_stacked_hv}')

        total_df = num_of_w.join(df_stacked_lm, on=series.name, how='left', validate='1:1')
        total_df = total_df.drop('count in doc_right')
        #print(total_df.shape)

        total_df = total_df.join(df_stacked_hv, on=series.name, how='left', validate='1:1')
        total_df = total_df.drop('count in doc_right')
        #print(total_df)
  

        #print(df_stacked_lm.shape, df_stacked_hv.shape)

        #total_df = df_stacked_lm.join(df_stacked_hv, on=df_series.name, how='outer')
        #print(total_df)

        return total_df

check = count_sentiment_w_in_doc(example_series)  
print(check)

shape: (410, 4)
┌──────────────────┬──────────────┬─────────────┬─────────────┐
│ 2009-05-07_18652 ┆ count in doc ┆ positive_lm ┆ positive_hv │
│ ---              ┆ ---          ┆ ---         ┆ ---         │
│ str              ┆ u32          ┆ bool        ┆ bool        │
╞══════════════════╪══════════════╪═════════════╪═════════════╡
│ weakness         ┆ 2            ┆ false       ┆ null        │
│ disaster         ┆ 1            ┆ false       ┆ false       │
│ moderate         ┆ 1            ┆ null        ┆ true        │
│ capital          ┆ 4            ┆ null        ┆ null        │
│ ease             ┆ 2            ┆ null        ┆ null        │
│ …                ┆ …            ┆ …           ┆ …           │
│ enhance          ┆ 1            ┆ true        ┆ true        │
│ suit             ┆ 2            ┆ null        ┆ true        │
│ clear            ┆ 3            ┆ null        ┆ null        │
│ forfeitures      ┆ 2            ┆ false       ┆ null        │
│ discretion       ┆ 1  

This is the func that counts a number of docs with at least one occurense of the input word. It takes hole dictionary whith dataframes and particular word as an input. Then checks how many cols in each df contain this word, sum up all resulst and return a single value for each word.

In [14]:
def in_all_doc_occurence(check_str, df_dict):
    count = 0
    for df in df_dict.values():
        count += df.select((pl.all() == check_str).sum() > 0).sum_horizontal()[0]
        
    return count

In [15]:
stringg = "a"
in_all_doc_occurence(stringg, df_dict)

0

This is the func that takes a df from count_sentiment_w_in_doc() and for each word in this df applies in_all_doc_occurence(), meaning that it counts all docs with at least one occurence of this word. The result of applying this func for each word in df is a series of integers showing counts for correcdonding word. This series augmented af col named "count in all docs"

In [16]:
def total_occurence_count(df_series, df_dict):
   check = count_sentiment_w_in_doc(df_series).lazy()

   counts = check.with_columns(pl.col(check.columns[0]).map_elements(lambda x: in_all_doc_occurence(x, df_dict)).alias("count in all docs")).collect()
   return counts

In [17]:
res_df = total_occurence_count(example_series, df_dict)
print(res_df)

shape: (410, 5)
┌──────────────────┬──────────────┬─────────────┬─────────────┬───────────────────┐
│ 2009-05-07_18652 ┆ count in doc ┆ positive_lm ┆ positive_hv ┆ count in all docs │
│ ---              ┆ ---          ┆ ---         ┆ ---         ┆ ---               │
│ str              ┆ u32          ┆ bool        ┆ bool        ┆ i64               │
╞══════════════════╪══════════════╪═════════════╪═════════════╪═══════════════════╡
│ complete         ┆ 3            ┆ null        ┆ null        ┆ 6495              │
│ acceptance       ┆ 8            ┆ null        ┆ true        ┆ 2967              │
│ treat            ┆ 5            ┆ null        ┆ true        ┆ 1014              │
│ tax              ┆ 39           ┆ null        ┆ null        ┆ 8451              │
│ unauthorized     ┆ 1            ┆ false       ┆ null        ┆ 2628              │
│ …                ┆ …            ┆ …           ┆ …           ┆ …                 │
│ benefited        ┆ 1            ┆ true        ┆ null      

This is the func that takes as an argument resulting df, wich was obtained from total_occurence_count(). Recal that this df contains information about one document (one col in df). This func calculate weighted and raw scores using precomputed metrics. Returns a list with weighted score and raw score. Weights are calculated based on this formula: 

$$
w_{i,j} = 
\begin{cases} 
\left(\frac{1 + \log(tf_{i,j})}{1 + \log(a)}\right) \log\left(\frac{N}{df_i}\right) & \text{if } tf_{i,j} \geq 1 \\
0 & \text{otherwise} 
\end{cases}
$$


N represents the total number of 10-Ks in the sample, dfi the number of documents containing at least one occurrence of the i th word, tfi,j the raw count of the i th word in the j th document, and a the average word count in the document.

The formula is from:

Loughran T., McDonald B. When is a liability not a liability? Textual analysis, dictionaries, and 10‐Ks //The Journal of finance. – 2011. – Т. 66. – №. 1. – С. 35-65.

In [18]:
def compute_score(res_df):

    res_df.lazy()

    res = res_df.with_columns((((1 + pl.col(res_df.columns[1]).log())/(1 + ln_average_word_count))*((total_num_of_rep/pl.col(res_df.columns[4])).log())).alias("metrics"))

    
    lm_positive = res.filter(pl.col(('positive_lm')) == True)
    w_pos_sen_lm = lm_positive.select(pl.col(res.columns[5])).sum().item()

    lm_negative = res.filter(pl.col(('positive_lm')) == False)
    w_neg_sen_lm = lm_negative.select(pl.col(res.columns[5])).sum().item()

    hv_positive = res.filter(pl.col(('positive_hv')) == True)
    w_pos_sen_hv = hv_positive.select(pl.col(res.columns[5])).sum().item()

    hv_negative = res.filter(pl.col(('positive_hv')) == False)
    w_neg_sen_hv = hv_negative.select(pl.col(res.columns[5])).sum().item()

    w_score_lm = w_pos_sen_lm - w_neg_sen_lm
    w_score_hv = w_pos_sen_hv - w_neg_sen_hv

    #lm_score = lm_positive.select(pl.col("count in doc")).sum().item() - lm_negative.select(pl.col("count in doc")).sum().item()
    #hv_score = hv_positive.select(pl.col("count in doc")).sum().item() - lm_negative.select(pl.col("count in doc")).sum().item()
    #print(lm_score, hv_score)

    #w_pos_sen = res.filter(pl.col('positive_lm') == True).select(pl.col(res.columns[5])).sum().item()
    #w_neg_sen = res.filter(pl.col('positive_lm') == False).select(pl.col(res.columns[5])).sum().item()
#
    #w_score = w_pos_sen - w_neg_sen
    #score = res.select(pl.col('positive')).sum().item()*2 - len(res)
#
    scores = [w_score_lm, w_score_hv]
    
    return scores

compute_score(res_df)

[-35.1818105242069, 3.862172798847496]

This is the function that takes polars series as an input, then concatinate all values in the single string. For this sitring, which is the initial report, this func calculates sentiment scores according to Harvard IV-4 and Loughran and McDonald Financial Sentiment Dictionaries. Perticulary, this function calculates polarity score which is defined by the following formula:

$$
Polarity = \frac{N_{\text{pos}} - N_{\text{neg}}}{N_{\text{pos}} + N_{\text{neg}}}
$$

In [19]:
hiv4 = ps.HIV4()
lm = ps.LM() 

def compute_pysentiment(series):

    doc_string = series.str.concat(" ", ignore_nulls=True)

    tokens_hiv4 = hiv4.tokenize(doc_string[0])
    tokens_lm = lm.tokenize(doc_string[0])

    score_hiv4 = hiv4.get_score(tokens_hiv4)
    score_lm = lm.get_score(tokens_lm)

  
    return score_lm["Polarity"], score_hiv4["Polarity"]

In [20]:
#Example of pysentiment2 working on example_series: first value is "Polsrity" score from Loughran-McDonald dictionary, and second one is from Harvard dictionary
lst = [1, 48.0303, 334, 'extended values starts from here:']

lst.extend(compute_pysentiment(example_series))

print(lst)

[1, 48.0303, 334, 'extended values starts from here:', -0.5077399372945203, 0.1573604059582399]


In [21]:
pattern = r"_(\d+)$"

def get_doc_len(df):
    string_with_len = df.columns[0]

    match = re.search(pattern, string_with_len)
    doc_len = match.group(1)

    return doc_len


General loop that iterates over dfs and over it columns. It uses predefined finctions to calculate scores and save it as parquet files. Each parquet file named with company name is a dataframe with col names as field dates, first row is weighted scores and second row is raw scores. 

In [109]:
for company_name, df in df_dict.items():
    counter = 0

    score_dict = {}

    for col in df:
        
        res_df = total_occurence_count(col, df_dict)
        print('res df is ready')
        doc_len = get_doc_len(res_df)
        scores = compute_score(res_df)

        scores.extend(compute_pysentiment(col))
        scores.append(doc_len)

        score_dict[col.name] = scores

    counter += 1
    
    df = pl.DataFrame(score_dict)
    
    print(f'Iteration for {company_name} is done successfully; \n {len(df_dict) - counter} remaining')

 # Determine the output directory and file name
    output_dir = os.path.join('data', '2013_reports')
    os.makedirs(output_dir, exist_ok=True)

    file_name_new = f"{company_name}_full_doc_sen_score_df.parquet"
    full_path = os.path.join(output_dir, file_name_new)
    full_path = os.path.normpath(full_path)

    print(f"Attempting to write to: {full_path}")

    # Write the DataFrame to Parquet
    df.write_parquet(full_path)

    print(f"{full_path} created successfully")
        
        

NameError: name 'total_occurence_count' is not defined